<a href="https://colab.research.google.com/github/seunghee0518/R_File/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%A7%88%EC%9D%B4%EB%8B%9D_%EA%B8%B0%EB%A7%90%EA%B3%BC%EC%A0%9C%EB%AC%BC(%EB%B0%95%EC%8A%B9%ED%9D%AC%2C202035_368335).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1번
* numcredits변수에 factor()함수 적용
* dummy함수 적용

In [ ]:
#german 데이터 불러오기
german_url = 'https://raw.githubusercontent.com/seunghee0518/R_File/main/%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%A7%88%EC%9D%B4%EB%8B%9D/germandata.txt'
german = read.table(german_url, header = T)
head(german)
attach(german)

In [ ]:
summary(numcredits)

In [ ]:
summary(factor(numcredits))

In [ ]:
german$numcredits = factor(german$numcredits)
german$residence = factor(german$residence)
german$residpeople = factor(german$residpeople)

In [ ]:
install.packages('dummy')
library(dummy)
dvar = c(4, 9, 10, 15, 17)
german2 = dummy(x = german[, dvar])
german2 = german2[, -c(10, 14, 17, 20, 24)]
german2 = cbind(german[, -dvar], german2)
for(i in 1:ncol(german2)) if(!is.numeric(german2[, i])) german2[, i]=as.numeric(german2[,i])
german2$y = ifelse(german$y == 'good', 1, 0)


In [ ]:
head(german2)

In [ ]:
names(german2)

In [ ]:
summary(german2$numcredits)

In [ ]:
type(german2$numcredits)

##2번

In [ ]:
a = 1-((2/6)^2 + (4/6)^2)
b = 1-((1/5)^2 + (4/5)^2)
c = 1-((5/7)^2 + (2/7)^2)

In [ ]:
a

In [ ]:
b

In [ ]:
c

In [ ]:
bc = 1-((6/12)^2 + (6/12)^2)
ca = 1-((7/13)^2 + (6/13)^2)
ab = 1-((3/11)^2 + (8/11)^2)

In [ ]:
a_bc = (6/18)*a + (12/18)*bc
a_bc

In [ ]:
b_ca = (5/18)*b + (13/18)*ca
b_ca

In [ ]:
c_ab = (7/18)*c + (11/18)*ab
c_ab

#3번

#5번 : 신경망
* 50%(종별25개) - 학습데이터
* seed : 368335
* 나머지로 종류분류 평가
* 신경망 구조도 출력

In [ ]:
#패키지
install.packages("neuralnet")
library(neuralnet)

In [ ]:
iris

In [ ]:
#목표변수 Species -> 수치형 변환
#setosa = 1
#versicolor = 2
#virginica = 3
iris$Species = ifelse(iris$Species == "setosa", 1, 
              +        ifelse(iris$Species == "versicolor", 2, 3))
iris$Species

In [ ]:
#train data
set.seed(368335)
i <- c(sample(1:50, 25), sample(51:100, 25), sample(101:150, 25))

#표준화
max.iris = apply(iris, 2, max)
min.iris = apply(iris, 2, min)

ziris = scale(iris, center = min.iris, scale = max.iris - min.iris)
ziris = as.data.frame(ziris)

#훈련/검증 데이터 분리
train <- ziris[i, ]
test <- ziris[-i, ]

In [ ]:
#신경망 학습
n = names(train)
form = as.formula(paste("Species ~ ", paste(n[!n %in% "Species"], collapse = "+")))
nn1 = neuralnet(form, data = train, hidden = c(5, 3), linear.output = F)
nn2 = neuralnet(form, data = train, hidden = c(3, 2), linear.output = F)

In [ ]:
install.packages("devtools")
library(devtools)
source('https://gist.githubusercontent.com/fawda123/7471137/raw/466c1474d0a505ff044412703516c34f1a4684a5/nnet_plot_update.r')
plot.nnet(nn1)

In [ ]:
#예측(첫번째 은닉층5개, 두번째 은닉층 3개)
pred.nn1 = compute(nn1, test)

#원래대로 복귀
pred1 = pred.nn1$net.result*(max(iris$Species)-min(iris$Species))+min(iris$Species)

#m1값미만 : 1(setosa)/ m2이상 3(virginica)
#경계라인 : (max-min)/3을 기준으로 분류
d = max(pred1)-min(pred1)
m1 = min(pred1)+(d/3)
m2 = max(pred1)-(d/3)

pred.nn2 = ifelse(pred1<m1, 1,
            +ifelse(pred1>= m1 & pred1 < m2, 2, 3))

In [ ]:
#오분류율
# 테스트 데이터의 5열(종)과 예측값 비교해서 틀린 개수 / 테스트 데이터의 갯수
sum(iris[-i, 5] != pred.nn2) / length(iris[-i, 5])

In [ ]:
#오분류표
table(iris[-i, 5], pred.nn2)

In [ ]:
#예측 (첫번째 은닉층3개, 두번째 은닉층 2개)
pred.nn2 = compute(nn2, test)

#원래대로 복귀
pred2 = pred.nn2$net.result*(max(iris$Species)-min(iris$Species))+min(iris$Species)

#m1값미만 : 1(setosa)/ m2이상 3(virginica)
#경계라인 : (max-min)/3을 기준으로 분류
d = max(pred2)-min(pred2)
m1 = min(pred2)+(d/3)
m2 = max(pred2)-(d/3)

pred.32 = ifelse(pred2<m1, 1,
            +ifelse(pred2>= m1 & pred2 < m2, 2, 3))

sum(iris[-i, 5] != pred.32) / length(iris[-i, 5])
table(iris[-i, 5], pred.32)

In [ ]:
plot.nnet(nn2)